# Redes residuales

Bienvenido a la primera tarea de esta semana. Vas a construir una red convolucional muy profunda, utilizando redes residuales (ResNets). En teoría, las redes muy profundas pueden representar funciones muy complejas, pero en la práctica son difíciles de entrenar. Las redes residuales, introducidas por [He et al.](https://arxiv.org/pdf/1512.03385.pdf), permiten entrenar redes mucho más profundas de lo que era posible anteriormente.

**Al final de esta tarea, será capaz de

- Implementar los bloques básicos de ResNets en una red neuronal profunda utilizando Keras
- Reunir estos bloques de construcción para implementar y entrenar una red neuronal de última generación para la clasificación de imágenes
- Implementar una conexión de salto en su red

Para esta tarea, utilizarás Keras. 

Antes de entrar en el problema, ejecute la celda de abajo para cargar los paquetes necesarios.

## Nota importante sobre el envío al AutoGrader

Antes de enviar su tarea al AutoGrader, por favor asegúrese de que no está haciendo lo siguiente:

1. 1. No ha añadido ninguna declaración _extra_ `print` en la tarea.
2. 2. No ha añadido ninguna celda de código _extra_ en la tarea.
3. No ha cambiado ningún parámetro de la función.
4. No está utilizando ninguna variable global dentro de sus ejercicios calificados. A menos que se le indique específicamente que lo haga, por favor absténgase de hacerlo y utilice las variables locales en su lugar.
5. 5. No está cambiando el código de asignación donde no es necesario, como la creación de variables _extra_.

Si hace algo de lo siguiente, obtendrá un error como `Grader no encontrado` (o similarmente inesperado) al enviar su tarea. Antes de pedir ayuda/depurar los errores de su tarea, compruebe esto primero. Si este es el caso, y no recuerda los cambios que ha realizado, puede obtener una nueva copia de la tarea siguiendo estas [instrucciones](https://www.coursera.org/learn/convolutional-neural-networks/supplement/DS4yP/h-ow-to-refresh-your-workspace).

Traducción realizada con la versión gratuita del traductor www.DeepL.com/Translator

## Table of Content

- [1 - Packages](#1)
- [2 - The Problem of Very Deep Neural Networks](#2)
- [3 - Building a Residual Network](#3)
    - [3.1 - The Identity Block](#3-1)
        - [Exercise 1 - identity_block](#ex-1)
    - [3.2 - The Convolutional Block](#3-2)
        - [Exercise 2 - convolutional_block](#ex-2)
- [4 - Building Your First ResNet Model (50 layers)](#4)
    - [Exercise 3 - ResNet50](#ex-3)
- [5 - Test on Your Own Image (Optional/Ungraded)](#5)
- [6 - Bibliography](#6)

<a name='1'></a>
## 1 - Packages

In [1]:
import tensorflow as tf
import numpy as np
import scipy.misc
from tensorflow.keras.applications.resnet_v2 import ResNet50V2
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet_v2 import preprocess_input, decode_predictions
from tensorflow.keras import layers
from tensorflow.keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D
from tensorflow.keras.models import Model, load_model
from resnets_utils import *
from tensorflow.keras.initializers import random_uniform, glorot_uniform, constant, identity
from tensorflow.python.framework.ops import EagerTensor
from matplotlib.pyplot import imshow

from test_utils import summary, comparator
import public_tests

%matplotlib inline

<a name='2'></a>
## 2 - El problema de las redes neuronales muy profundas

La semana pasada, construiste tus primeras redes neuronales convolucionales: primero manualmente con numpy, y luego usando Tensorflow y Keras. 

En los últimos años, las redes neuronales se han vuelto mucho más profundas, con redes de última generación que han pasado de tener unas pocas capas (por ejemplo, AlexNet) a más de cien capas.

* La principal ventaja de una red muy profunda es que puede representar funciones muy complejas. También puede aprender características en muchos niveles diferentes de abstracción, desde bordes (en las capas menos profundas, más cercanas a la entrada) hasta características muy complejas (en las capas más profundas, más cercanas a la salida). 

* Sin embargo, utilizar una red más profunda no siempre ayuda. Un gran obstáculo para entrenarlas es la desaparición de los gradientes: las redes muy profundas suelen tener una señal de gradiente que llega a cero rápidamente, lo que hace que el descenso de gradiente sea prohibitivamente lento.

* Más concretamente, durante el descenso de gradiente, al retropropagar desde la última capa hasta la primera, se multiplica por la matriz de pesos en cada paso, y por tanto el gradiente puede disminuir exponencialmente hasta llegar a cero (o, en casos raros, crecer exponencialmente y "explotar", al ganar valores muy grandes). 

* Durante el entrenamiento, se puede ver que la magnitud (o norma) del gradiente para las capas más superficiales disminuye a cero muy rápidamente a medida que avanza el entrenamiento, como se muestra a continuación: 

<img src="images/vanishing_grad_kiank.png" style="width:450px;height:220px;">
<caption><center> <u> <font color='green'> <b>Figure 1</b> </u><font color='green'>  : <b>Vanishing gradient</b> <br> La velocidad de aprendizaje disminuye muy rápidamente para las capas menos profundas a medida que la red se entrena </center></caption>.

<center>¡No hay que preocuparse! Ahora vas a resolver este problema construyendo una Red Residual!</center>

<a name='3'></a>
## 3 - Building a Residual Network

In ResNets, a "shortcut" or a "skip connection" allows the model to skip layers:  

<img src="images/skip_connection_kiank.png" style="width:650px;height:200px;">
<caption><center> <u> <font color='green'> <b>Figure 2</b> </font color="green"></u>  : Un bloque ResNet mostrando una conexión de salto <br> </center></caption>

La imagen de la izquierda muestra la "ruta principal" a través de la red. La imagen de la derecha añade un acceso directo a la ruta principal. Apilando estos bloques ResNet unos sobre otros, se puede formar una red muy profunda. 

En la conferencia se mencionó que tener bloques ResNet con el atajo también hace muy fácil que uno de los bloques aprenda una función de identidad. Esto significa que se pueden apilar bloques ResNet adicionales con poco riesgo de perjudicar el rendimiento del conjunto de entrenamiento.  
    
En este sentido, también hay pruebas de que la facilidad de aprendizaje de una función de identidad explica el notable rendimiento de las ResNets, incluso más que las conexiones de salto ayudan a los gradientes de fuga.

En una ResNet se utilizan dos tipos principales de bloques, dependiendo principalmente de si las dimensiones de entrada/salida son iguales o diferentes. Se van a implementar ambos: el "bloque de identidad" y el "bloque convolucional".

<a name='3-1'></a>
### 3.1 - The Identity Block

El bloque de identidad es el bloque estándar utilizado en las ResNets, y corresponde al caso en el que la activación de entrada (digamos $a^{[l]}$) tiene la misma dimensión que la activación de salida (digamos $a^{[l+2]}$). Para explicar los diferentes pasos de lo que ocurre en el bloque de identidad de una ResNet, he aquí un diagrama alternativo que muestra los pasos individuales:

<img src="images/idblock2_kiank.png" style="width:650px;height:150px;">
<caption><center> <u> <font color='green'> <b>Figure 3</b><b>Identity block.</b> Skip connection "skips over" 2 layers. </font color='green'>  </u> </center></caption>

El camino superior es el "camino de acceso directo". El camino inferior es el "camino principal". En este diagrama, observe los pasos CONV2D y ReLU en cada capa. Para acelerar el entrenamiento, se ha añadido un paso BatchNorm. No se preocupe por que esto sea complicado de implementar - ¡verá que BatchNorm es sólo una línea de código en Keras! 

En este ejercicio, implementarás una versión ligeramente más potente de este bloque de identidad, en la que la conexión de salto "salta" 3 capas ocultas en lugar de 2 capas. Se ve así: 

<img src="images/idblock3_kiank.png" style="width:650px;height:150px;">
    <caption><center> <u> <font color='green'> <b>Figure 4</b> </u><font color='green'>  : <b>Identity block.</b> Skip connection "skips over" 3 layers.</center></caption>

Estos son los pasos individuales:

Primer componente de la ruta principal: 
- El primer CONV2D tiene filtros $F_1$ de forma (1,1) y un paso de (1,1). Su relleno es "válido". Usa 0 como semilla para la inicialización uniforme aleatoria: `kernel_initializer = initializer(seed=0)`. 
- El primer BatchNorm está normalizando el eje de `canales`.
- A continuación, aplicar la función de activación ReLU. Esto no tiene hiperparámetros. 

Segundo componente de la ruta principal:
- El segundo CONV2D tiene $F_2$ filtros de forma $(f,f)$ y un stride de (1,1). Su relleno es "igual". Usa 0 como semilla para la inicialización uniforme aleatoria: `kernel_initializer = initializer(seed=0)`.
- El segundo BatchNorm está normalizando el eje de `canales`.
- A continuación, aplicar la función de activación ReLU. Esto no tiene hiperparámetros.

Tercer componente de la ruta principal:
- El tercer CONV2D tiene filtros $F_3$ de forma (1,1) y un stride de (1,1). Su relleno es "válido". Usa 0 como semilla para la inicialización uniforme aleatoria: `kernel_initializer = initializer(seed=0)`. 
- El tercer BatchNorm está normalizando el eje de `canales`.
- Observe que no hay **ninguna** función de activación ReLU en este componente. 

Final step: 
- The `X_shortcut` and the output from the 3rd layer `X` are added together.
- **Hint**: The syntax will look something like `Add()([var1,var2])`
- Then apply the ReLU activation function. This has no hyperparameters. 

<a name='ex-1'></a>
### Exercise 1 - identity_block

Implemente el bloque de identidad de ResNet. El primer componente de la ruta principal ya está implementado. En primer lugar, debes leer estos documentos con atención para asegurarte de que entiendes lo que ocurre. Luego, implemente el resto. 
- Para implementar el paso Conv2D: [Conv2D](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Conv2D)
- To implement BatchNorm: [BatchNormalization](https://www.tensorflow.org/api_docs/python/tf/keras/layers/BatchNormalization) `BatchNormalization(axis = 3)(X, training = training)`. Si el entrenamiento se establece en Falso, sus pesos no se actualizan con los nuevos ejemplos. Es decir, cuando el modelo se utiliza en modo de predicción.
- For the activation, use:  `Activación('relu')(X)`
- Para añadir el valor pasado por el atajo: [Add](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Add)

Hemos añadido el argumento inicializador a nuestras funciones. Este parámetro recibe una función inicializadora como las incluidas en el paquete [tensorflow.keras.initializers](https://www.tensorflow.org/api_docs/python/tf/keras/initializers) o cualquier otro inicializador personalizado. Por defecto será [random_uniform](https://www.tensorflow.org/api_docs/python/tf/keras/initializers/RandomUniform)

Recuerda que estas funciones aceptan un argumento `seed` que puede ser cualquier valor que desees, pero que en este cuaderno debe establecerse a 0 para **finalidades de clasificación**.

 Aquí es donde realmente se utiliza el poder de la API Funcional para crear un camino de acceso directo: 

In [2]:
# UNQ_C1
# GRADED FUNCTION: identity_block

def identity_block(X, f, filters, training=True, initializer=random_uniform):
    """
    Implementación del bloque de identidad definido en la figura 4
    
    Argumentos:
    X -- tensor de entrada de forma (m, n_H_prev, n_W_prev, n_C_prev)
    f -- número entero, que especifica la forma de la ventana del CONV medio para la ruta principal
    filters -- lista python de enteros, que define el número de filtros en las capas de la CONV de la ruta principal
    training -- Verdadero: Comportarse en modo de entrenamiento
                Falso: Comportarse en modo de inferencia
    inicializador -- para configurar los pesos iniciales de una capa. Es igual al inicializador uniforme aleatorio
    
    Devuelve:
    X -- salida del bloque de identidad, tensor de forma (m, n_H, n_W, n_C)
    """
    
    # Retrieve Filters
    F1, F2, F3 = filters
    
    # Save the input value. You'll need this later to add back to the main path. 
    X_shortcut = X
    
    # First component of main path
    X = Conv2D(filters            = F1, 
               kernel_size        = 1, 
               strides            = (1,1), 
               padding            = 'valid', 
               kernel_initializer = initializer(seed=0)
               )(X)
    X = BatchNormalization(axis = 3)(X, training = training) # Default axis
    X = Activation('relu')(X)
    
    ### START CODE HERE
    ## Second component of main path (≈3 lines)
    ## Set the padding = 'same'
    X = Conv2D(filters            = F2, 
               kernel_size        =  f, 
               strides            = (1,1) ,
               padding            = "same",
               kernel_initializer = initializer( seed = 0 )
               )(X)
    X = BatchNormalization(axis = 3)(X, training=training)
    X = Activation('relu')(X)

    ## Third component of main path (≈2 lines)
    ## Set the padding = 'valid'
    X = Conv2D(filters            = F3, 
               kernel_size        = 1, 
               strides            = (1, 1), 
               padding            = 'valid', 
               kernel_initializer = initializer(seed=0)
               )(X)
    X = BatchNormalization(axis = 3)(X, training=training)
    
    ## Final step: Add shortcut value to main path, and pass it through a RELU activation (≈2 lines)
    X = Add()([X_shortcut,X])
    X = Activation('relu')(X)
    ### END CODE HERE<

    return X

In [3]:
np.random.seed(1)
X1 = np.ones((1, 4, 4, 3)) * -1
X2 = np.ones((1, 4, 4, 3)) * 1
X3 = np.ones((1, 4, 4, 3)) * 3

X = np.concatenate((X1, X2, X3), axis = 0).astype(np.float32)

A3 = identity_block(X, f=2, filters=[4, 4, 3],
                   initializer=lambda seed=0:constant(value=1),
                   training=False)
print('\033[1mWith training=False\033[0m\n')
A3np = A3.numpy()
print(np.around(A3.numpy()[:,(0,-1),:,:].mean(axis = 3), 5))
resume = A3np[:,(0,-1),:,:].mean(axis = 3)
print(resume[1, 1, 0])

print('\n\033[1mWith training=True\033[0m\n')
np.random.seed(1)
A4 = identity_block(X, f=2, filters=[3, 3, 3],
                   initializer=lambda seed=0:constant(value=1),
                   training=True)
print(np.around(A4.numpy()[:,(0,-1),:,:].mean(axis = 3), 5))

public_tests.identity_block_test(identity_block)

With training=False

[[[  0.        0.        0.        0.     ]
  [  0.        0.        0.        0.     ]]

 [[192.7123  192.7123  192.7123   96.85615]
  [ 96.85615  96.85615  96.85615  48.92808]]

 [[578.1369  578.1369  578.1369  290.56845]
  [290.56845 290.56845 290.56845 146.78423]]]
96.85615

With training=True

[[[0.      0.      0.      0.     ]
  [0.      0.      0.      0.     ]]

 [[0.40739 0.40739 0.40739 0.40739]
  [0.40739 0.40739 0.40739 0.40739]]

 [[4.99991 4.99991 4.99991 3.25948]
  [3.25948 3.25948 3.25948 2.40739]]]
All tests passed!


**Expected value**

```
With training=False

[[[  0.        0.        0.        0.     ]
  [  0.        0.        0.        0.     ]]

 [[192.71234 192.71234 192.71234  96.85617]
  [ 96.85617  96.85617  96.85617  48.92808]]

 [[578.1371  578.1371  578.1371  290.5685 ]
  [290.5685  290.5685  290.5685  146.78426]]]
96.85617

With training=True

[[[0.      0.      0.      0.     ]
  [0.      0.      0.      0.     ]]

 [[0.40739 0.40739 0.40739 0.40739]
  [0.40739 0.40739 0.40739 0.40739]]

 [[4.99991 4.99991 4.99991 3.25948]
  [3.25948 3.25948 3.25948 2.40739]]]
```

<a name='3-2'></a>
### 3.2 - The Convolutional Block

El "bloque convolucional" de ResNet es el segundo tipo de bloque. Puede utilizar este tipo de bloque cuando las dimensiones de entrada y salida no coinciden. La diferencia con el bloque de identidad es que hay una capa CONV2D en la ruta de acceso: 

<img src="images/convblock_kiank.png" style="width:650px;height:150px;">
<caption><center> <u> <font color='green'> <b>Figure 4</b>   : <b>Convolutional block</b> </font color='green'></u></center></caption>

* La capa CONV2D en la ruta de acceso directo se utiliza para cambiar el tamaño de la entrada $x$ a una dimensión diferente, de modo que las dimensiones coincidan en la adición final necesaria para añadir el valor de acceso directo de nuevo a la ruta principal. (Esto juega un papel similar al de la matriz $W_s$ discutida en la conferencia). 
* Por ejemplo, para reducir la altura y la anchura de las dimensiones de activación en un factor de 2, se puede utilizar una convolución 1x1 con un paso de 2. 
* La capa CONV2D en la ruta de acceso no utiliza ninguna función de activación no lineal. Su papel principal es simplemente aplicar una función lineal (aprendida) que reduce la dimensión de la entrada, de modo que las dimensiones coincidan para el paso de adición posterior. 
* Al igual que en el ejercicio anterior, el argumento adicional `inicializador` es necesario para la graduación, y se ha establecido por defecto a [glorot_uniform](https://www.tensorflow.org/api_docs/python/tf/keras/initializers/GlorotUniform)

Los detalles del bloque convolucional son los siguientes. 

Primer componente de la ruta principal:
- El primer CONV2D tiene filtros $F_1$ de forma (1,1) y un stride de (s,s). Su relleno es "válid". Utiliza 0 como semilla `glorot_uniform` `kernel_initializer = initializer(seed=0)`.
- El primer BatchNorm es normalizar el eje `canales`.
- A continuación, aplicar la función de activación ReLU. Esto no tiene hiperparámetros. 

Segundo componente de la ruta principal:
- El segundo CONV2D tiene $F_2$ filtros de forma (f,f) y un stride de (1,1). Su relleno es "same".  Usa 0 como semilla `glorot_uniforme` `kernel_initializer = initializer(seed=0)`.
- El segundo BatchNorm es normalizar el eje `canales`.
- A continuación, aplicar la función de activación ReLU. Esto no tiene hiperparámetros. 

Tercer componente de la ruta principal:
- El tercer CONV2D tiene filtros $F_3$ de forma (1,1) y un stride de (1,1). Su relleno es "válid".  Utiliza 0 como semilla `glorot_uniforme` `kernel_initializer = initializer(seed=0)`.
- El third BatchNorm is normalizing the 'channels' axis. Note that there is no ReLU activation function in this component. 

Camino de acceso directo:
- El CONV2D tiene filtros $F_3$ de forma (1,1) y un stride de (s,s). Su relleno es "válid".  Usa 0 como semilla `glorot_uniforme` `kernel_initializer = initializer(seed=0)`.
- El BatchNorm está normalizando el eje de `canales`. 

Paso final: 
- Se suman los valores del atajo y del camino principal.
- Luego se aplica la función de activación ReLU. Esto no tiene hiperparámetros. 
 
<a name='ex-2'></a>    
### Exercise 2 - convolutional_block
    
Implementa el bloque convolucional. El primer componente de la ruta principal ya está implementado; ¡entonces es tu turno de implementar el resto! Como antes, utiliza siempre 0 como semilla para la inicialización aleatoria, para asegurar la consistencia con el clasificador.
- Conv2D](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Conv2D)
- BatchNormalization](https://www.tensorflow.org/api_docs/python/tf/keras/layers/BatchNormalization) (axis: Entero, el eje que debe ser normalizado (típicamente el eje de características)) `BatchNormalization(axis = 3)(X, training = training)`. Si el entrenamiento se establece en False, sus pesos no se actualizan con los nuevos ejemplos. Es decir, cuando el modelo se utiliza en modo de predicción.
- Para la activación, utilice:  `Activación('relu')(X)`
- [Añadir](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Add)
    
Hemos añadido el argumento inicializador a nuestras funciones. Este parámetro recibe una función inicializadora como las incluidas en el paquete [tensorflow.keras.initializers](https://www.tensorflow.org/api_docs/python/tf/keras/initializers) o cualquier otro inicializador personalizado. Por defecto será [random_uniform](https://www.tensorflow.org/api_docs/python/tf/keras/initializers/RandomUniform)

Recuerda que estas funciones aceptan un argumento `seed` que puede ser cualquier valor que desees, pero que en este cuaderno debe establecerse en 0 por **propósitos de clasificación**.

In [4]:
def convolutional_block(X, f, filters, s = 2, training=True, initializer=glorot_uniform):
    """
    Implementation of the convolutional block as defined in Figure 4
    
    Arguments:
    X -- input tensor of shape (m, n_H_prev, n_W_prev, n_C_prev)
    f -- integer, specifying the shape of the middle CONV's window for the main path
    filters -- python list of integers, defining the number of filters in the CONV layers of the main path
    s -- Integer, specifying the stride to be used
    training -- True: Behave in training mode
                False: Behave in inference mode
    initializer -- to set up the initial weights of a layer. Equals to Glorot uniform initializer, 
                   also called Xavier uniform initializer.
    
    Returns:
    X -- output of the convolutional block, tensor of shape (n_H, n_W, n_C)
    """
    
    # Retrieve Filters
    F1, F2, F3 = filters
    
    # Save the input value
    X_shortcut = X


    ##### MAIN PATH #####
    
    # First component of main path glorot_uniform(seed=0)
    X = Conv2D(filters = F1, kernel_size = 1, strides = (s, s), padding='valid', kernel_initializer = initializer(seed=0))(X)
    X = BatchNormalization(axis = 3)(X, training=training)
    X = Activation('relu')(X)

    ### START CODE HERE
    
    ## Second component of main path (≈3 lines)
    X = Conv2D(filters = F2, kernel_size = f, strides = (1, 1), padding='same', kernel_initializer = initializer(seed=0))(X)
    X = BatchNormalization(axis = 3)(X, training=training)
    X = Activation('relu')(X)

    ## Third component of main path (≈2 lines)
    X = Conv2D(filters = F3, kernel_size = 1, strides = (1, 1), padding='valid', kernel_initializer = initializer(seed=0))(X)
    X = BatchNormalization(axis = 3)(X, training=training)
    
    ##### SHORTCUT PATH ##### (≈2 lines)
    X_shortcut = Conv2D(filters = F3, kernel_size = 1, strides = (s, s), padding='valid', kernel_initializer = initializer(seed=0))(X_shortcut)
    X_shortcut = BatchNormalization(axis = 3)(X_shortcut, training=training)
    
    ### END CODE HERE

    # Final step: Add shortcut value to main path (Use this order [X, X_shortcut]), and pass it through a RELU activation
    X = Add()([X, X_shortcut])
    X = Activation('relu')(X)
    
    return X


In [5]:
# UNQ_C2
# GRADED FUNCTION: convolutional_block

def convolutional_block(X, f, filters, s = 2, training=True, initializer=glorot_uniform):
    """
    Implementación del bloque convolucional definido en la Figura 4
    
    Argumentos:
    X -- tensor de entrada de forma (m, n_H_prev, n_W_prev, n_C_prev)
    f -- entero, que especifica la forma de la ventana de la CONV del medio para el camino principal
    filters -- lista de enteros en python, que define el número de filtros en las capas de la CONV de la ruta principal
    s -- Número entero, que especifica la zancada que se utilizará
    training -- Verdadero: Comportarse en modo de entrenamiento
                Falso: Comportarse en modo de inferencia
    inicializador -- para configurar los pesos iniciales de una capa. Equivale al inicializador uniforme de Glorot, 
                   también llamado inicializador uniforme de Xavier.
    
    Devuelve:
    X -- salida del bloque convolucional, tensor de forma (m, n_H, n_W, n_C)
    """
    
    # Retrieve Filters
    F1, F2, F3 = filters
    
    # Save the input value
    X_shortcut = X


    ##### MAIN PATH #####
    
    # First component of main path glorot_uniform(seed=0)
    X = Conv2D(filters            = F1, 
               kernel_size        = 1, 
               strides            = (s, s), 
               padding            = 'valid', 
               kernel_initializer = initializer(seed=0)
               )(X)
    X = BatchNormalization(axis = 3)(X, training=training)
    X = Activation('relu')(X)

    ### START CODE HERE
    """
    ## Second component of main path (≈3 lines)
    X =  Conv2D(filters           = F2, 
               kernel_size        = f, 
               strides            = (1, 1), 
               padding            = 'same', 
               kernel_initializer = initializer(seed=0)
               )(X)
    X = BatchNormalization(axis = 3)(X, training=training)
    X = Activation('relu')(X)
 

    ## Third component of main path (≈2 lines)
    X = Conv2D(filters            = F3, 
               kernel_size        = 1, 
               strides            = (1, 1), 
               padding            = 'valid', 
               kernel_initializer = initializer(seed=0)
               )(X)
    X = BatchNormalization(axis = 3)(X, training=training) 
    
    ##### SHORTCUT PATH ##### (≈2 lines)
    X_shortcut = Conv2D(filters            = F3, 
                        kernel_size        = 1, 
                        strides            = (s, s), 
                        padding            = 'valid', 
                        kernel_initializer = initializer(seed=0)
                        )(X_shortcut)
    X_shortcut = BatchNormalization(axis = 3)(X_shortcut, training=training)
    
    ### END CODE HERE

    # Final step: Add shortcut value to main path (Use this order [X, X_shortcut]), and pass it through a RELU activation
    X = Add()([X, X_shortcut])
    X = Activation('relu')(X)
    
    return X
    """
    X = Conv2D(filters = F2, kernel_size = f,strides = (1, 1),padding='same',kernel_initializer = initializer(seed=0))(X)
    X = BatchNormalization(axis = 3)(X, training=training)
    X = Activation('relu')(X)

    ## Third component of main path (≈2 lines)
    X = Conv2D(filters = F3, kernel_size = 1, strides = (1, 1), padding='valid', kernel_initializer = initializer(seed=0))(X)
    X = BatchNormalization(axis = 3)(X, training=training)
    
    ##### SHORTCUT PATH ##### (≈2 lines)
    X_shortcut = Conv2D(filters = F3, kernel_size = 1, strides = (s, s), padding='valid', kernel_initializer = initializer(seed=0))(X_shortcut)
    X_shortcut = BatchNormalization(axis = 3)(X_shortcut, training=training)
    
    ### END CODE HERE

    # Final step: Add shortcut value to main path (Use this order [X, X_shortcut]), and pass it through a RELU activation
    X = Add()([X, X_shortcut])
    X = Activation('relu')(X)
    
    return X

In [6]:
from outputs import convolutional_block_output1, convolutional_block_output2
np.random.seed(1)
#X = np.random.randn(3, 4, 4, 6).astype(np.float32)
X1 = np.ones((1, 4, 4, 3)) * -1
X2 = np.ones((1, 4, 4, 3)) * 1
X3 = np.ones((1, 4, 4, 3)) * 3

X = np.concatenate((X1, X2, X3), axis = 0).astype(np.float32)

A = convolutional_block(X, f = 2, filters = [2, 4, 6], training=False)

assert type(A) == EagerTensor, "Use only tensorflow and keras functions"
assert tuple(tf.shape(A).numpy()) == (3, 2, 2, 6), "Wrong shape."
assert np.allclose(A.numpy(), convolutional_block_output1), "Wrong values when training=False."
print(A[0])

B = convolutional_block(X, f = 2, filters = [2, 4, 6], training=True)
assert np.allclose(B.numpy(), convolutional_block_output2), "Wrong values when training=True."

print('\033[92mAll tests passed!')


AssertionError: Wrong values when training=False.

**Expected value**

```
tf.Tensor(
[[[0.         0.66683817 0.         0.         0.88853896 0.5274254 ]
  [0.         0.65053666 0.         0.         0.89592844 0.49965227]]

 [[0.         0.6312079  0.         0.         0.8636247  0.47643146]
  [0.         0.5688321  0.         0.         0.85534114 0.41709304]]], shape=(2, 2, 6), dtype=float32)
```

<a name='4'></a>  
## 4 - Construir su primer modelo ResNet (50 capas)

Ahora tiene los bloques necesarios para construir una ResNet muy profunda. La siguiente figura describe en detalle la arquitectura de esta red neuronal. "ID BLOCK" en el diagrama significa "Identity block", y "ID BLOCK x3" significa que debe apilar 3 bloques de identidad.

<img src="images/resnet_kiank.png" style="width:850px;height:150px;">
<caption><center> <u> <font color='green'> <b>Figure 5</b> </u></font color='green'>  : <b>ResNet-50 model</b> </center></caption>

Los detalles de este modelo ResNet-50 son
- El padding de cero rellena la entrada con un relleno de (3,3)
- Etapa 1:
    - La Convolución 2D tiene 64 filtros de forma (7,7) y utiliza un stride de (2,2). 
    - Se aplica BatchNorm al eje "canales" de la entrada.
    - Se aplica la activación ReLU.
    - MaxPooling utiliza una ventana (3,3) y un stride (2,2).
- Etapa 2:
    - El bloque convolucional utiliza tres conjuntos de filtros de tamaño [64,64,256], "f" es 3 y "s" es 1.
    - Los 2 bloques de identidad utilizan tres conjuntos de filtros de tamaño [64,64,256], y "f" es 3.
- Fase 3:
    - El bloque convolucional utiliza tres conjuntos de filtros de tamaño [128,128,512], "f" es 3 y "s" es 2.
    - Los 3 bloques de identidad utilizan tres conjuntos de filtros de tamaño [128,128,512] y "f" es 3.
- Etapa 4:
    - El bloque convolucional utiliza tres conjuntos de filtros de tamaño [256, 256, 1024], "f" es 3 y "s" es 2.
    - Los 5 bloques de identidad utilizan tres conjuntos de filtros de tamaño [256, 256, 1024] y "f" es 3.
- Etapa 5:
    - El bloque convolucional utiliza tres conjuntos de filtros de tamaño [512, 512, 2048], "f" es 3 y "s" es 2.
    - Los 2 bloques de identidad utilizan tres conjuntos de filtros de tamaño [512, 512, 2048] y "f" es 3.
- El Average Pooling 2D utiliza una ventana de forma (2,2).
- La capa "flatten" no tiene ningún hiperparámetro.
- La capa totalmente conectada (densa) reduce su entrada al número de clases utilizando una activación softmax.


Traducción realizada con la versión gratuita del traductor www.DeepL.com/Translator

    
<a name='ex-3'></a>      
### Exercise 3 - ResNet50 
    
Implementar la ResNet con 50 capas descrita en la figura anterior. Hemos implementado las etapas 1 y 2. Por favor, implemente el resto. (La sintaxis para implementar las Etapas 3-5 debería ser bastante similar a la de la Etapa 2) Asegúrate de seguir la convención de nomenclatura del texto anterior. 

Tendrás que utilizar esta función: 
- Media de la agrupación [ver referencia](https://www.tensorflow.org/api_docs/python/tf/keras/layers/AveragePooling2D)

Aquí hay otras funciones que utilizamos en el código de abajo:
- Conv2D: [Ver referencia](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Conv2D)
- BatchNorm: [Ver referencia](https://www.tensorflow.org/api_docs/python/tf/keras/layers/BatchNormalization) (axis: Entero, el eje que debe ser normalizado (típicamente el eje de características))
- Relleno de cero: [Ver referencia](https://www.tensorflow.org/api_docs/python/tf/keras/layers/ZeroPadding2D)
- Max pooling: [Ver referencia](https://www.tensorflow.org/api_docs/python/tf/keras/layers/MaxPool2D)
- Capa totalmente conectada: [Ver referencia](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Dense)
- Adición: [Ver referencia](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Add)

Traducción realizada con la versión gratuita del traductor www.DeepL.com/Translator

In [ ]:
# UNQ_C3
# GRADED FUNCTION: ResNet50

def ResNet50(input_shape = (64, 64, 3), classes = 6):
    """
    Stage-wise implementation of the architecture of the popular ResNet50:
    CONV2D -> BATCHNORM -> RELU -> MAXPOOL -> CONVBLOCK -> IDBLOCK*2 -> CONVBLOCK -> IDBLOCK*3
    -> CONVBLOCK -> IDBLOCK*5 -> CONVBLOCK -> IDBLOCK*2 -> AVGPOOL -> FLATTEN -> DENSE 

    Arguments:
    input_shape -- shape of the images of the dataset
    classes -- integer, number of classes

    Returns:
    model -- a Model() instance in Keras
    """
    
    # Define the input as a tensor with shape input_shape
    X_input = Input(input_shape)

    
    # Zero-Padding
    X = ZeroPadding2D((3, 3))(X_input)
    
    # Stage 1
    X = Conv2D(64, (7, 7), strides = (2, 2), kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3)(X)
    X = Activation('relu')(X)
    X = MaxPooling2D((3, 3), strides=(2, 2))(X)

    # Stage 2
    X = convolutional_block(X, f = 3, filters = [64, 64, 256], s = 1)
    X = identity_block(X, 3, [64, 64, 256])
    X = identity_block(X, 3, [64, 64, 256])

    ### START CODE HERE
    
    ## Stage 3 (≈4 lines)
    X = convolutional_block(X, f = 3, filters = [128, 128, 512], s = 2)
    X = identity_block(X, 3, [128, 128, 512]) 
    X = identity_block(X, 3, [128, 128, 512]) 
    X = identity_block(X, 3, [128, 128, 512])  
    
    ## Stage 4 (≈6 lines)
    X = convolutional_block(X, f = 3, filters = [256, 256, 1024], s = 2)
    X = identity_block(X, 3, [256, 256, 1024])
    X = identity_block(X, 3, [256, 256, 1024])
    X = identity_block(X, 3, [256, 256, 1024])
    X = identity_block(X, 3, [256, 256, 1024])
    X = identity_block(X, 3, [256, 256, 1024]) 

    ## Stage 5 (≈3 lines)
    X = convolutional_block(X, f = 3, filters = [512, 512, 2048], s = 2)
    X = identity_block(X, 3, [512, 512, 2048])  
    X = identity_block(X, 3, [512, 512, 2048])  

    ## AVGPOOL (≈1 line). Use "X = AveragePooling2D(...)(X)"
    X = AveragePooling2D(pool_size=(2, 2))(X)  
    
    ### END CODE HERE

    # output layer
    X = Flatten()(X)
    X = Dense(classes, activation='softmax', kernel_initializer = glorot_uniform(seed=0))(X)
    
    
    # Create model
    model = Model(inputs = X_input, outputs = X)

    return model

Run the following code to build the model's graph. If your implementation is incorrect, you'll know it by checking your accuracy when running `model.fit(...)` below.

In [ ]:
model = ResNet50(input_shape = (64, 64, 3), classes = 6)
print(model.summary())

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 64, 64, 3)]  0           []                               
                                                                                                  
 zero_padding2d (ZeroPadding2D)  (None, 70, 70, 3)   0           ['input_1[0][0]']                
                                                                                                  
 conv2d_20 (Conv2D)             (None, 32, 32, 64)   9472        ['zero_padding2d[0][0]']         
                                                                                                  
 batch_normalization_20 (BatchN  (None, 32, 32, 64)  256         ['conv2d_20[0][0]']              
 ormalization)                                                                                

In [ ]:
from outputs import ResNet50_summary

model = ResNet50(input_shape = (64, 64, 3), classes = 6)

comparator(summary(model), ResNet50_summary)


All tests passed!


As shown in the Keras Tutorial Notebook, prior to training a model, you need to configure the learning process by compiling the model.

In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

The model is now ready to be trained. The only thing you need now is a dataset!

Let's load your old friend, the SIGNS dataset.

<img src="images/signs_data_kiank.png" style="width:450px;height:250px;">
<caption><center> <u> <font color='green'> <b>Figure 6</b> </u><font color='green'>  : <b>SIGNS dataset</b> </center></caption>


In [ ]:
X_train_orig, Y_train_orig, X_test_orig, Y_test_orig, classes = load_dataset()

# Normalize image vectors
X_train = X_train_orig / 255.
X_test = X_test_orig / 255.

# Convert training and test labels to one hot matrices
Y_train = convert_to_one_hot(Y_train_orig, 6).T
Y_test = convert_to_one_hot(Y_test_orig, 6).T

print ("number of training examples = " + str(X_train.shape[0]))
print ("number of test examples = " + str(X_test.shape[0]))
print ("X_train shape: " + str(X_train.shape))
print ("Y_train shape: " + str(Y_train.shape))
print ("X_test shape: " + str(X_test.shape))
print ("Y_test shape: " + str(Y_test.shape))

number of training examples = 1080
number of test examples = 120
X_train shape: (1080, 64, 64, 3)
Y_train shape: (1080, 6)
X_test shape: (120, 64, 64, 3)
Y_test shape: (120, 6)


Run the following cell to train your model on 10 epochs with a batch size of 32. On a GPU, it should take less than 2 minutes. 

In [ ]:
model.fit(X_train, Y_train, epochs = 10, batch_size = 32)

Epoch 1/10
20/34 [================>.............] - ETA: 1:12 - loss: 2.1689 - accuracy: 0.3609

KeyboardInterrupt: 

**Expected Output**:

```
Epoch 1/10
34/34 [==============================] - 1s 34ms/step - loss: 1.9241 - accuracy: 0.4620
Epoch 2/10
34/34 [==============================] - 2s 57ms/step - loss: 0.6403 - accuracy: 0.7898
Epoch 3/10
34/34 [==============================] - 1s 24ms/step - loss: 0.3744 - accuracy: 0.8731
Epoch 4/10
34/34 [==============================] - 2s 44ms/step - loss: 0.2220 - accuracy: 0.9231
Epoch 5/10
34/34 [==============================] - 2s 57ms/step - loss: 0.1333 - accuracy: 0.9583
Epoch 6/10
34/34 [==============================] - 2s 52ms/step - loss: 0.2243 - accuracy: 0.9444
Epoch 7/10
34/34 [==============================] - 2s 48ms/step - loss: 0.2913 - accuracy: 0.9102
Epoch 8/10
34/34 [==============================] - 1s 30ms/step - loss: 0.2269 - accuracy: 0.9306
Epoch 9/10
34/34 [==============================] - 2s 46ms/step - loss: 0.1113 - accuracy: 0.9630
Epoch 10/10
34/34 [==============================] - 2s 57ms/step - loss: 0.0709 - accuracy: 0.9778
```

The exact values could not match, but don't worry about that. The important thing that you must see is that the loss value decreases, and the accuracy increases for the firsts 5 epochs.

Let's see how this model (trained on only two epochs) performs on the test set.

In [ ]:
preds = model.evaluate(X_test, Y_test)
print ("Loss = " + str(preds[0]))
print ("Test Accuracy = " + str(preds[1]))

**Expected Output**:

<table>
    <tr>
        <td>
            <b>Test Accuracy</b>
        </td>
        <td>
           >0.80
        </td>
    </tr>

</table>

For the purposes of this assignment, you've been asked to train the model for ten epochs. You can see that it performs well. The online grader will only run your code for a small number of epochs as well. Please go ahead and submit your assignment. 

After you have finished this official (graded) part of this assignment, you can also optionally train the ResNet for more iterations, if you want. It tends to get much better performance when trained for ~20 epochs, but this does take more than an hour when training on a CPU. 

Using a GPU, this ResNet50 model's weights were trained on the SIGNS dataset. You can load and run the trained model on the test set in the cells below. It may take ≈1min to load the model. Have fun! 

In [ ]:
pre_trained_model = tf.keras.models.load_model('resnet50.h5')

In [ ]:
preds = pre_trained_model.evaluate(X_test, Y_test)
print ("Loss = " + str(preds[0]))
print ("Test Accuracy = " + str(preds[1]))

**Congratulations** on finishing this assignment! You've now implemented a state-of-the-art image classification system! Woo hoo! 

ResNet50 is a powerful model for image classification when it's trained for an adequate number of iterations. Hopefully, from this point, you can use what you've learned and apply it to your own classification problem to perform state-of-the-art accuracy.

<font color = 'blue'>

**What you should remember**:

- Very deep "plain" networks don't work in practice because vanishing gradients make them hard to train.  
- Skip connections help address the Vanishing Gradient problem. They also make it easy for a ResNet block to learn an identity function. 
- There are two main types of blocks: The **identity block** and the **convolutional block**. 
- Very deep Residual Networks are built by stacking these blocks together.

<a name='5'></a>  
## 5 - Test on Your Own Image (Optional/Ungraded)

If you wish, you can also take a picture of your own hand and see the output of the model. To do this:
    1. Click on "File" in the upper bar of this notebook, then click "Open" to go on your Coursera Hub.
    2. Add your image to this Jupyter Notebook's directory, in the "images" folder
    3. Write your image's name in the following code
    4. Run the code and check if the algorithm is right! 

In [ ]:
img_path = 'images/my_image.jpg'
img = image.load_img(img_path, target_size=(64, 64))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = x/255.0
print('Input image shape:', x.shape)
imshow(img)
prediction = pre_trained_model.predict(x)
print("Class prediction vector [p(0), p(1), p(2), p(3), p(4), p(5)] = ", prediction)
print("Class:", np.argmax(prediction))


Even though the model has high accuracy, it might be performing poorly on your own set of images. Notice that, the shape of the pictures, the lighting where the photos were taken, and all of the preprocessing steps can have an impact on the performance of the model. Considering everything you have learned in this specialization so far, what do you think might be the cause here?

*Hint*: It might be related to some distributions. Can you come up with a potential solution ?

You can also print a summary of your model by running the following code.

In [ ]:
pre_trained_model.summary()

<a name='6'></a>  
## 6 - Bibliography

This notebook presents the ResNet algorithm from He et al. (2015). The implementation here also took significant inspiration and follows the structure given in the GitHub repository of Francois Chollet: 

- Kaiming He, Xiangyu Zhang, Shaoqing Ren, Jian Sun - [Deep Residual Learning for Image Recognition (2015)](https://arxiv.org/abs/1512.03385)
- Francois Chollet's GitHub repository: https://github.com/fchollet/deep-learning-models/blob/master/resnet50.py
